<a href="https://colab.research.google.com/github/MansurPro/Prompt2Clip/blob/main/Prompt2Clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies

In [ ]:
!pip install torch==2.5.1 accelerate==1.1.1 hf_transfer==0.1.8 sentencepiece==0.2.0 imageio==2.36.0 imageio-ffmpeg==0.5.1
!pip install git+https://github.com/huggingface/transformers@30335093276212ce74938bdfd85bfd5df31a668a
!pip install git+https://github.com/huggingface/diffusers@99c0483b67427de467f11aa35d54678fd36a7ea2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.36.1
    Uninstalling imageio-2.36.1:
      Successfully uninstalled imageio-2.36.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
  Cloning https://github.com/huggingface/transformers (to revision 30335093276212ce74938bdfd85bfd5df31a668a) to /tmp/pip-req-build-1oulbfqh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1oulbfqh
  Running command git rev-parse -q --verify 'sha^30335093276212ce74938bdfd85bfd5df31a668a'
  Running command git fetch -q https://github.com/huggingface/transformers 30

In [ ]:
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["HF_HOME"] = "/content/models"

# Model Download and Initialization

In [ ]:
import torch
from diffusers import MochiPipeline

# Define paths
MODEL_PATH = "/content/models"
OUTPUTS_PATH = "/content/outputs"

# Ensure directories exist
os.makedirs(MODEL_PATH, exist_ok=True)
os.makedirs(OUTPUTS_PATH, exist_ok=True)

# Download the model
revision = "83359d26a7e2bbe200ecbfda8ebff850fd03b545"
pipe = MochiPipeline.from_pretrained(
    "genmo/mochi-1-preview",
    torch_dtype=torch.bfloat16,
    revision=revision,
    cache_dir=MODEL_PATH
)
pipe.enable_model_cpu_offload()
pipe.enable_vae_tiling()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

transformer/config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

(…)pytorch_model-00001-of-00005.safetensors:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00005.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Generate Video

In [ ]:
from diffusers.utils import export_to_video
from pathlib import Path
import string
import time

def slugify(prompt):
    for char in string.punctuation:
        prompt = prompt.replace(char, "")
    prompt = prompt.replace(" ", "_")
    prompt = prompt[:230]
    mp4_name = str(int(time.time())) + "_" + prompt + ".mp4"
    return mp4_name

prompt = "A cinematic 4K depiction of a fierce warrior in mid-battle, showcasing dynamic martial arts moves with precision and agility, surrounded by the chaos of war, with dust, fire, and clashing weapons creating a dramatic and intense atmosphere."
negative_prompt = ""
num_inference_steps = 200
guidance_scale = 4.5
num_frames = 19

frames = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    num_frames=num_frames,
).frames[0]

# Save video
mp4_name = slugify(prompt)
export_to_video(frames, Path(OUTPUTS_PATH) / mp4_name)
print(f"Video saved at: {Path(OUTPUTS_PATH) / mp4_name}")

# View the Generated Video

In [ ]:
from IPython.display import Video

video_path = f"{OUTPUTS_PATH}/{mp4_name}"
Video(video_path, embed=True)